In [2]:
import numpy as np
data = np.load('lab2_data.npz', allow_pickle=True)['data']

In [55]:
#phoneHMMs = np.load('lab2_models_onespkr.npz', allow_pickle=True)['phoneHMMs'].item()
phoneHMMs = np.load('lab2_models_all.npz', allow_pickle=True)['phoneHMMs'].item()

In [56]:
phoneHMMs["f"]

{'name': 'f',
 'startprob': array([1., 0., 0., 0.]),
 'transmat': array([[0.8074276, 0.1925724, 0.       , 0.       ],
        [0.       , 0.7835307, 0.2164693, 0.       ],
        [0.       , 0.       , 0.4751177, 0.5248824],
        [0.       , 0.       , 0.       , 1.       ]]),
 'means': array([[ 3.756838e+02, -1.892536e+02, -9.660455e+01,  4.465488e+01,
          2.823588e+01, -4.992766e-01,  4.529542e-02, -2.909808e+01,
          1.526023e+01, -3.026565e+01,  2.106727e+01, -3.956148e+01,
          2.572369e+01],
        [ 5.441120e+02, -1.801021e+02, -1.472212e+02, -1.085416e+01,
         -2.093026e+01, -1.813961e+01, -1.523612e+01, -5.690070e+01,
         -1.829125e+01, -4.970216e+01,  9.754507e+00, -5.453812e+01,
          1.968687e+01],
        [ 7.118344e+02,  2.438120e+02, -5.496016e+01,  8.645602e+00,
         -1.945088e+02,  3.470830e+01,  3.832467e+01, -7.212926e+00,
         -4.773862e+01, -2.433857e+01,  1.006693e+02, -2.232076e+01,
          7.655536e+01]]),
 'covars':

In [57]:
list(sorted(phoneHMMs.keys()))

['ah',
 'ao',
 'ay',
 'eh',
 'ey',
 'f',
 'ih',
 'iy',
 'k',
 'n',
 'ow',
 'r',
 's',
 'sil',
 'sp',
 't',
 'th',
 'uw',
 'v',
 'w',
 'z']

In [58]:
import pandas as pd
# Convert to DataFrame
df = pd.DataFrame(phoneHMMs["f"]["covars"])
df

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,19210.64,9451.407,7170.096,6573.493,7262.504,8466.096,10791.600,11358.400,11268.830,11928.65,11280.83,10306.790,9663.794
1,10238.43,10038.210,4539.494,4773.204,5569.295,7163.819,7795.367,8738.427,9528.196,10172.64,9130.49,8674.652,8326.857
2,29271.92,16966.650,12817.220,10414.770,38556.290,13704.490,12345.750,10492.110,17783.310,33074.34,21727.96,12487.230,12336.300


In [59]:
pd.DataFrame(phoneHMMs["f"]["means"])

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,375.6838,-189.2536,-96.60455,44.654880,28.23588,-0.499277,0.045295,-29.098080,15.26023,-30.26565,21.067270,-39.56148,25.72369
1,544.1120,-180.1021,-147.22120,-10.854160,-20.93026,-18.139610,-15.236120,-56.900700,-18.29125,-49.70216,9.754507,-54.53812,19.68687
2,711.8344,243.8120,-54.96016,8.645602,-194.50880,34.708300,38.324670,-7.212926,-47.73862,-24.33857,100.669300,-22.32076,76.55536


In [60]:
prondict = {} 
prondict['o'] = ['ow']
prondict['z'] = ['z', 'iy', 'r', 'ow']
prondict['1'] = ['w', 'ah', 'n']
prondict['2'] = ['t', 'uw']
prondict['3'] = ['th', 'r', 'iy']
prondict['4'] = ['f', 'ao', 'r']
prondict['5'] = ['f', 'ay', 'v']
prondict['6'] = ['s', 'ih', 'k', 's']
prondict['7'] = ['s', 'eh', 'v', 'ah', 'n']
prondict['8'] = ['ey', 't']
prondict['9'] = ['n', 'ay', 'n']
isolated = {}
for digit in prondict.keys():
    isolated[digit] = ['sil'] + prondict[digit] + ['sil']

In [132]:
def concatTwoHMMs(hmm1, hmm2):
   """ Concatenates 2 HMM models

    Args:
       hmm1, hmm2: two dictionaries with the following keys:
           name: phonetic or word symbol corresponding to the model
           startprob: M+1 array with priori probability of state
           transmat: (M+1)x(M+1) transition matrix
           means: MxD array of mean vectors
           covars: MxD array of variances

    D is the dimension of the feature vectors
    M is the number of emitting states in each HMM model (could be different for each)

    Output
       dictionary with the same keys as the input but concatenated models:
          startprob: K+1 array with priori probability of state
          transmat: (K+1)x(K+1) transition matrix
             means: KxD array of mean vectors
            covars: KxD array of variances

    K is the sum of the number of emitting states from the input models
   
    Example:
       twoHMMs = concatHMMs(phoneHMMs['sil'], phoneHMMs['ow'])
    """
   #startprob
   M1 = len(hmm1["startprob"])
   M2 = len(hmm2["startprob"])
   startprob_conc = np.zeros((M1+M2-1))
   for i in range(M1+M2-1):
      if i<M1-1:
         startprob_conc[i]=hmm1["startprob"][i]
      else:
         startprob_conc[i]=hmm1["startprob"][-1]*hmm2["startprob"][i-M2]

         
   #transmat
   A = hmm1["transmat"][:-1, :-1] #n-1 x n-1 matrix, skip last row and column
   B1 = hmm1["transmat"][:-1, -1]  #last row minus the 1 at the bottom in first mat.
   B2 = hmm2["startprob"]
   B = B1.reshape(-1, 1) @ B2.reshape(1, -1) #reshape to go from (3,) x (4,) to (3,1) x (1,4)
   C = np.zeros((M2,M1-1))
   D = hmm2["transmat"]
   top_row = np.concatenate((A, B), axis=1)
   bottom_row = np.concatenate((C, D), axis=1)
   transmat_conc = np.concatenate((top_row, bottom_row), axis=0)

   #mean
   top = hmm1["means"]
   bottom = hmm2["means"]
   mean_conc = np.concatenate((top, bottom), axis=0)

   #covars
   top = hmm1["covars"]
   bottom = hmm2["covars"]
   covars_conc = np.concatenate((top, bottom), axis=0)

   return {"startprob": startprob_conc,
           "transmat": transmat_conc,
           "mean": mean_conc,
           "covars": covars_conc,
           "name": hmm1["name"] +"_"+hmm2["name"]
           }
concatTwoHMMs(phoneHMMs['sil'], phoneHMMs['ow'])

{'startprob': array([1., 0., 0., 0., 0., 0., 0.]),
 'transmat': array([[0.8833106 , 0.1166894 , 0.        , 0.        , 0.        ,
         0.        , 0.        ],
        [0.        , 0.9164465 , 0.08355348, 0.        , 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.802877  , 0.197123  , 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.8148277 , 0.1851723 ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.8436188 ,
         0.1563813 , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.8662466 , 0.1337534 ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 1.        ]]),
 'mean': array([[ 1.929856e+02, -1.658209e+01, -2.290160e+01,  3.132330e+01,
          4.658450e+01, -1.151792e+00,  1.339505e+01,  6.989146e+01,
          6.058731e+01,  1.809310e+01,  1.219584e+01, -2.165135e+01,

In [96]:
A = np.ones((3,3))
B= 2*np.ones((3,4))
C = 3*np.ones((4,3))
D = 4*np.ones((4,4))

top_row = np.concatenate((A, B), axis=1)
bottom_row = np.concatenate((C, D), axis=1)

# Vertical concatenation of the results
result_matrix = np.concatenate((top_row, bottom_row), axis=0)
print(result_matrix)

[[1. 1. 1. 2. 2. 2. 2.]
 [1. 1. 1. 2. 2. 2. 2.]
 [1. 1. 1. 2. 2. 2. 2.]
 [3. 3. 3. 4. 4. 4. 4.]
 [3. 3. 3. 4. 4. 4. 4.]
 [3. 3. 3. 4. 4. 4. 4.]
 [3. 3. 3. 4. 4. 4. 4.]]


In [106]:
result_matrix[:, -1]

array([2., 2., 2., 4., 4., 4., 4.])